In [65]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip


class VideoProcessor(object):
    def __init__(self, video_path, landmark_path, output_folder, extract_audio):
        # path of the video file
        self.video_path = video_path
        self.landmarks_path = landmark_path
        self.extract_audio = extract_audio
        self.frames_folder = os.path.join(output_folder, "frames")
        self.audios_folder = os.path.join(output_folder, "audios")

        if not os.path.exists(output_folder):
            os.mkdir(output_folder)
        if not os.path.exists(self.audios_folder):
            os.mkdir(self.audios_folder)
        if not os.path.exists(self.frames_folder):
            os.mkdir(self.frames_folder)

    def preprocess(self, seq_len=30, target_resolution=(224, 224)):
        """
        extract frames and audio from the video,
        store the cropped frames and audio file in the output folders
        seq_len: how many frames will be extracted from the video.
                  Considering all videos from this dataset have similar duration
                  video_duration = seq_len / fps
        target_resolution: (desired_height, desired_width) of the facial frame extracted
        """
        print("self video path : "+self.video_path)
        video = VideoFileClip(self.video_path, audio=self.extract_audio, target_resolution=target_resolution)
        if self.extract_audio:
            video.audio.write_audiofile(os.path.join(self.audios_folder, "audio.wav"))

        times = list(np.arange(0, video.duration, video.duration/seq_len))
        if len(times) < seq_len:
            times.append(video.duration)
        times = times[:seq_len]

        # extract 2D points from csv
        data = np.genfromtxt(self.landmarks_path, delimiter=',')[1:]
        lm_times = [int(np.ceil(t)) for t in list(np.arange(0, len(data), len(data) / seq_len))]
        if len(lm_times) < seq_len:
            lm_times.append(len(data) - 1)
        lm_times = lm_times[:seq_len]
        index_x = (298, 366)
        index_y = (366, 433)
        landmarks_2d_x = [data[t, index_x[0] - 1:index_x[1] - 1] * (1 / 1280) for t in lm_times]
        landmarks_2d_y = [data[t, index_y[0] - 1:index_y[1]] * (1 / 720) for t in lm_times]

        for i, t in enumerate(times):
            img = cv2.cvtColor(video.get_frame(t), cv2.COLOR_BGR2RGB)
            # extract roi from landmarks and crop
            xs, ys = landmarks_2d_x[i], landmarks_2d_y[i]
            bottom = int(max(ys) * img.shape[0])
            right = int(max(xs) * img.shape[1])
            top = int(min(ys) * img.shape[0])
            left = int(min(xs) * img.shape[1])

            cropped = cv2.resize(img[top:bottom, left:right, :], target_resolution)
            cv2.imwrite(os.path.join(self.frames_folder, "frame_{0:.2f}.jpg".format(t)), cropped)

        print("Video duration {} seconds. Extracted {} frames".format(video.duration, len(times)))

In [66]:
import numpy as np
import glob
import os
import argparse
from tqdm import tqdm
import librosa



def preprocess_video(dataset_folder):
    output_folder = os.path.join(dataset_folder, "preprocessed")
    landmark_folder = os.path.join(dataset_folder, "landmarks")

    for each_actor in glob.glob(os.path.join(dataset_folder, "Actor*")):
        actor_name = os.path.basename(each_actor)
        output_actor_folder = os.path.join(output_folder, actor_name)
        print("output actor folder : "+output_actor_folder)
        if not os.path.exists(output_actor_folder):
            os.makedirs(output_actor_folder)

        for each_video in glob.glob(os.path.join(each_actor, "*.mp4")):
            print("each video :"+each_video)
            name = each_video.split("/")[-1].split("-")[0]
            print("name : "+name)
            if int(name) == 2:
                continue
            video_name = os.path.basename(each_video)
            print('video name : '+video_name)
            landmark_path = os.path.join(landmark_folder, video_name[:-4] + '.csv')
            print('landmark path : '+landmark_path)
            frames_folder = os.path.join(output_actor_folder, video_name)
            print("frames folder : "+frames_folder)
            if not os.path.exists(frames_folder):
                os.mkdir(frames_folder)
            print("frame folder : "+frames_folder)    
            video_processor = VideoProcessor(video_path=each_video, landmark_path=landmark_path,
                                             output_folder=frames_folder, extract_audio=True)
            video_processor.preprocess(seq_len=30, target_resolution=(224, 224))

In [67]:
if __name__ == "__main__":
    #parser = argparse.ArgumentParser()
    #parser.add_argument('/content/drive/MyDrive/Datasetvideo/', type=str, help='dataset directory', default='RAVDESS')
    #args = parser.parse_args()

    print("Processing videos...")
    preprocess_video('/content/drive/MyDrive/Datasetvideo/')

   

    print("Preprocessed dataset located in ", os.path.join('/content/drive/MyDrive/Datasetvideo/', 'preprocessed'))

Processing videos...
output actor folder : /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/01-01-01-01-01-01-01.mp4
name : 01
video name : 01-01-01-01-01-01-01.mp4
landmark path : /content/drive/MyDrive/Datasetvideo/landmarks/01-01-01-01-01-01-01.csv
frames folder : /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-01-01-01-01-01.mp4
frame folder : /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-01-01-01-01-01.mp4
self video path : /content/drive/MyDrive/Datasetvideo/Actor_01/01-01-01-01-01-01-01.mp4
[MoviePy] Writing audio in /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-01-01-01-01-01.mp4/audios/audio.wav


100%|██████████| 74/74 [00:00<00:00, 1731.86it/s]

[MoviePy] Done.


Video duration 3.35 seconds. Extracted 30 frames
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/01-01-01-01-02-01-01.mp4
name : 01
video name : 01-01-01-01-02-01-01.mp4
landmark path : /content/drive/MyDrive/Datasetvideo/landmarks/01-01-01-01-02-01-01.csv
frames folder : /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-01-01-02-01-01.mp4
frame folder : /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-01-01-02-01-01.mp4
self video path : /content/drive/MyDrive/Datasetvideo/Actor_01/01-01-01-01-02-01-01.mp4
[MoviePy] Writing audio in /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-01-01-02-01-01.mp4/audios/audio.wav


100%|██████████| 73/73 [00:00<00:00, 1693.40it/s]

...

[MoviePy] Done.


Video duration 3.31 seconds. Extracted 30 frames
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-01-01-01-01-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-01-01-02-01-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-01-01-01-02-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-01-01-01-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-01-01-02-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/01-01-08-02-02-02-01.mp4
name : 01
video name : 01-01-08-02-02-02-01.mp4
landmark path : /content/drive/MyDrive/Datasetvideo/landmarks/01-01-08-02-02-02-01.csv
frames folder : /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-08-02-02-02-01.mp4
frame folder : /content/drive/MyDrive/Datasetvideo/preprocessed/Actor_01/01-01-08-02-02-02-01.mp4
self video path : /content/drive/MyDrive/Datasetvideo/Actor_01/01-01-08-02-02

100%|██████████| 73/73 [00:00<00:00, 1850.27it/s]

[MoviePy] Done.


Video duration 3.31 seconds. Extracted 30 frames
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-01-01-02-02-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-01-02-01-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-02-01-02-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-01-02-02-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-02-01-01-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-02-02-01-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-02-02-02-02-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-03-02-01-02-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-03-01-02-01-01.mp4
name : 02
each video :/content/drive/MyDrive/Datasetvideo/Actor_01/02-01-03-01-01-02-01.mp4
name : 02
each video :/content/drive/MyDr